In [45]:
import pygame
import random
import math
import colorsys
from pygame.locals import *
import matplotlib.pyplot as plt

idx = [0, 0, 0] #global variable index - the index of last two selected polygon corner points, to limit neighbors

In [46]:
#colorscheme from here: https://beltoforion.de/en/recreational_mathematics/chaos_game.php
#remember for future use! rainbow patter is cool for hydrodynamical stuff

In [64]:
def mark_pixel(surface, pos, pcol):
    #takes in a pygame surface, a position, and a color
    #each vertex is given a point on the color wheel
    #each point is given a color based on the vertex of target corner point
    #if colors fall on each other, addative using pygame add
    col = surface.get_at(pos) #originally, colors do not exsist (nan) and are plottes as black
    surface.set_at(pos, (min(col[0] + pcol[0]/8, 255),
                         min(col[1] + pcol[1]/9, 255),
                         min(col[2] + pcol[2]/10, 255)))
    #this sets the color as the minimum of white (255) and the original color plus 1/x the color of the target vertex
    #this works because even though the points won't touch, they overlap on pixels
    #change colors for cool rgb values

In [65]:
def random_point_index(p):
    #next point cannont be a neighbor of chosen point, otherwise chosen randomly
    global idx #get gloabl variable!!!!!!
    idx[2] = idx[1]#shift things along the list
    idx[1] = idx[0] #same here
    dst1 = abs(idx[1] - idx[2]) #this should be non-zero so that these two indexes are not the same!

    while True:
        idx[0] = random.randint(0, len(p) - 1)  # Purely random selection, the rest is just applying constraints
        dst = abs(idx[0] - idx[1]) #just distance between the points - if dst==0, then this is the same point!
        if dst1 == 0 and (dst == 1 or dst == len(p) - 1): 
            #first case takes care of the initial (0,0). the rest makes sure that the next vertex is not directly
            #before or after
            continue #if bad, then get a new number
        else:
            break #if good, then give the number!

    return idx[0]

In [66]:
def init_polygon(width, height, n):
    
    #looked this math up lol
    delta_angle = 360/n
    r = width/2 - 10
    
    p = []

    for i in range(0, n):
        
        angle = (180 + i*delta_angle) * math.pi / 180
        #just gives color wheel - thanks google! lol
        color = colorsys.hsv_to_rgb((i*delta_angle)/360, 0.8, 1)
        
        p.append(((width/2 + r*math.sin(angle),
                   height/2 + r*math.cos(angle)),
                  (int(color[0]*255), int(color[1]*255), int(color[2]*255)))) #appens ((x,y),hsvcolorcoords)
    return p

In [67]:
def main(width, height, n, r):
    pygame.init()
    surface = pygame.display.set_mode((width, height))
    #create pixelxpixel surface

    p = init_polygon(width, height, n)
    #creates pixel coords for the polygon

    x, y = (100, 100)
    for i in range(0, width*height*15): #change the 3 for lots of fun
        i = random_point_index(p) #gives the index of the next point
        x += (p[i][0][0] - x) * r #gets x coord from x and then adds by scaled value. if r-.5 then halfway game
        y += (p[i][0][1] - y) * r #same here yay
        #r should be #n/(n+3)

        mark_pixel(surface, (int(x), int(y)), p[i][1]) #marks pixel with color (updated!)
        #int is so we dont get half pixels
        if i % 100 == 0:
            pygame.display.update()
            
            
        #so my computer doesn't die when help
        for event in pygame.event.get():
            if event.type == QUIT:
                pygame.quit()
                return

    pygame.quit()


In [ ]:
# if __name__ == "__main__": #just to make sure lol
main(1000, 1000, 7, .5)